In [5]:
import os
import tensorflow as tf
import numpy as np

In [9]:
TRAIN_PATH = "../data_train/processed_audio/train.npz"
TEST_PATH = "../data_train/processed_audio/test.npz"

data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_train, tf.float32), y_train))
    .cache()
    .shuffle(buffer_size=len(X_train))
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_dataset = (
    tf.data.Dataset.from_tensor_slices((tf.cast(X_test, tf.float32), y_test))
    .cache()
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

# Compresión de modelos a Tensorflow Lite

## Cuantización completa a enteros de 8 bits

In [21]:
# Rutas
MODEL_H5_PATH = "../models/not_compressed_audio/model_CNNA.h5"
TFLITE_PATH = "../models/compressed_int8/model_CNNA.tflite"


# Dataset representativo para calibración de int8
def representative_dataset():
    for i in range(100):
        sample = X_train[i].astype(np.float32)[np.newaxis, :]
        yield [sample]

# Cargar el modelo H5 entrenado
model = tf.keras.models.load_model(MODEL_H5_PATH)

# Prepara el convertidor TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
converter.representative_dataset = representative_dataset

# Convierte y guarda
tflite_quant = converter.convert()
with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_quant)

print("Modelo TFLite guardado en:", TFLITE_PATH)

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpghu8ejoh\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpghu8ejoh\assets
c:\Users\Jaime\anaconda3\envs\tf_env\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Modelo TFLite guardado en: ../models/compressed_int8/model_CNNA.tflite


Script para ver las operaciones  que realiza  el modelo

In [12]:
TFLITE_PATH = '../models/compressed_int8/model_NN.tflite'

interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

# Verificar tipo de entrada/salida
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")

# Verificar operaciones utilizadas
ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.int8'> 
Tipo de salida: <class 'numpy.int8'> 

Operaciones utilizadas en el modelo TFLite:
FULLY_CONNECTED
SOFTMAX


Script para pasa  los modelos  a un archivo binario

In [ ]:
# Rutas
INT8_TFLITE_PATH = "../models/compressed_int8/model_NN.tflite"
CPP_PATH = "../V_int8/main/model.cpp"
HEADER_PATH = "../V_int8/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(INT8_TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.hpp`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_int8/main/model.cpp
Header guardado en ../V_int8/main/model.hpp


## Conversión a TFLite sin cuantizar

In [41]:
MODEL_H5_PATH = "../models/not_compressed_audio/model_CNNA.h5"
FLOAT_TFLITE_PATH = "../models/compressed_float32/model_CNNA.tflite"

model = tf.keras.models.load_model(MODEL_H5_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()
with open(FLOAT_TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print(f"Modelo cuantizado")

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmp7owq2o4s\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmp7owq2o4s\assets


Modelo cuantizado


In [37]:
TFLITE_PATH = '../models/compressed_float32/model_CNN.tflite'

interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")

ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.float32'> 
Tipo de salida: <class 'numpy.float32'> 

Operaciones utilizadas en el modelo TFLite:
CONV_2D
EXPAND_DIMS
FULLY_CONNECTED
MAX_POOL_2D
PACK
RESHAPE
SHAPE
SOFTMAX
STRIDED_SLICE


In [42]:
# Rutas
FLOAT_TFLITE_PATH = "../models/compressed_float32/model_CNNA.tflite"
CPP_PATH = "../V_float/main/model.cpp"
HEADER_PATH = "../V_float/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(FLOAT_TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.h`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_float/main/model.cpp
Header guardado en ../V_float/main/model.hpp


## Versiones MFCC


In [43]:
# Rutas de los datos procesados
TRAIN_PATH = "../data_train/processed_MFCC/train.npz"
TEST_PATH = "../data_train/processed_MFCC/test.npz"

# Cargar datos
data_train = np.load(TRAIN_PATH)
data_test = np.load(TEST_PATH)

X_train, y_train = data_train["X"], data_train["y"]
X_test, y_test = data_test["X"], data_test["y"]

In [ ]:
MODEL_H5_PATH = "../models/not_compressed_MFCC/model_CNN.h5"
TFLITE_PATH = "../models/compressed_MFCC/model_CNN.tflite"

model = tf.keras.models.load_model(MODEL_H5_PATH)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]

tflite_model = converter.convert()
with open(TFLITE_PATH, "wb") as f:
    f.write(tflite_model)

print("Modelo cuantizado")

INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpj1h896o9\assets


INFO:tensorflow:Assets written to: C:\Users\Jaime\AppData\Local\Temp\tmpj1h896o9\assets


Modelo cuantizado


In [74]:
TFLITE_PATH = "../models/compressed_MFCC/model_CNNA.tflite"
interpreter = tf.lite.Interpreter(model_path=TFLITE_PATH)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(f"Tipo de entrada: {input_details[0]['dtype']} ")
print(f"Tipo de salida: {output_details[0]['dtype']} ")
print("Formato de entrada esperado:", model.input_shape)

ops_set = set()
for idx in range(len(interpreter._get_ops_details())):
    op_name = interpreter._get_ops_details()[idx]['op_name']
    ops_set.add(op_name)

print("\nOperaciones utilizadas en el modelo TFLite:")
for op in sorted(ops_set):
    print(f"{op}")


Tipo de entrada: <class 'numpy.float32'> 
Tipo de salida: <class 'numpy.float32'> 
Formato de entrada esperado: (None, 49, 13)

Operaciones utilizadas en el modelo TFLite:
ADD
CONV_2D
FULLY_CONNECTED
MAX_POOL_2D
MUL
PACK
RESHAPE
SHAPE
SOFTMAX
STRIDED_SLICE


In [45]:
# Rutas
TFLITE_PATH = "../models/compressed_MFCC/model_CNN.tflite"
CPP_PATH = "../V_MFCC/main/model.cpp"
HEADER_PATH = "../V_MFCC/main/model.hpp"

# Cargar modelo TFLite cuantizado
with open(TFLITE_PATH, "rb") as f:
    tflite_model = f.read()

#Convertir a array de bytes en formato C++
cpp_array = ", ".join(str(b) for b in tflite_model)

# 📄 Generar código C++ (`model.cpp`)
cpp_code = f"""#include "model.hpp"

alignas(8) const unsigned char model_tflite[] = {{
    {cpp_array}
}};

const int model_tflite_len = {len(tflite_model)};
"""

# Guardar `model.cpp`
with open(CPP_PATH, "w") as f:
    f.write(cpp_code)

# 📄 Generar código C++ (`model.h`)
header_code = """#ifndef MODEL_H_
#define MODEL_H_

extern const unsigned char model_tflite[];
extern const int model_tflite_len;

#endif  // MODEL_H_
"""

# Guardar `model.h`
with open(HEADER_PATH, "w") as f:
    f.write(header_code)

print(f"Modelo exportado a C++ en {CPP_PATH}")
print(f"Header guardado en {HEADER_PATH}")

Modelo exportado a C++ en ../V_MFCC/main/model.cpp
Header guardado en ../V_MFCC/main/model.hpp


## Comparación de técnicas

Código para comparar tamaños

In [31]:
def model_size(n_bytes):
    for unit in ['B','KB','MB','GB','TB']:
        if n_bytes < 1024.0:
            return f"{n_bytes:.5f} {unit}"
        n_bytes /= 1024.0

# Rutas a tus modelos
keras_path = "../models/not_compressed_audio/model_NN.h5"
tflite_int8_path = "../models/compressed_int8/model_NN.tflite"
tflite_float_path = "../models/compressed_float32/model_NN.tflite"

size_keras  = os.path.getsize(keras_path)
size_tflite_int8 = os.path.getsize(tflite_int8_path)
size_tflite_float = os.path.getsize(tflite_float_path)

print("Keras model:", model_size(size_keras))
print("TFLite int model:", model_size(size_tflite_int8))
print("TFLite float model:", model_size(size_tflite_float))

Keras model: 5.89761 MB
TFLite int model: 503.15625 KB
TFLite float model: 1.95738 MB
